In [134]:
!pip install aif360

Defaulting to user installation because normal site-packages is not writeable


Gender as sensitive features

In [135]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



df=pd.read_csv('./recruitmentdataset-2022-1.3.csv')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

df['gender'] = df['gender'].map({'male': 1, 'female': 2, 'other': 3})
df['nationality']=df['nationality'].map({'Dutch': 1, 'Belgian': 2,'German':3})


# Replace boolean values with 0 and 1
df = pd.get_dummies(df.drop('Id',axis=1))

df = df.replace({False: 0, True: 1})

# Min-max scaling of columns age, ind-university_grade and ind-languages
df['age'] = (df['age'] - df['age'].min()) / (df['age'].max() - df['age'].min())
df['ind-university_grade'] = (df['ind-university_grade'] - df['ind-university_grade'].min()) / (df['ind-university_grade'].max() - df['ind-university_grade'].min())
df['ind-languages'] = (df['ind-languages'] - df['ind-languages'].min()) / (df['ind-languages'].max() - df['ind-languages'].min())

X = df.drop(['decision'], axis=1)
y = df['decision']

# One-hot encode the target variable (one column for each class)
y = y.replace({False: 0, True: 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [136]:
print(X_train)

      gender       age  nationality  ind-university_grade  ind-debateclub  \
3994       1  0.545455            1              0.727273               0   
423        2  0.545455            1              0.484848               0   
2991       1  0.636364            1              0.575758               1   
1221       2  0.454545            1              0.545455               1   
506        1  0.454545            1              0.545455               1   
...      ...       ...          ...                   ...             ...   
1130       2  0.363636            1              0.636364               0   
1294       3  0.636364            1              0.363636               0   
860        2  0.545455            1              0.636364               0   
3507       1  0.454545            3              0.606061               0   
3174       2  0.636364            2              0.666667               0   

      ind-programming_exp  ind-international_exp  ind-entrepeneur_exp  \
39

In [137]:

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

clf = SVC(kernel='rbf', C=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print('Accuracy on test data:', accuracy_score(y_test, y_pred))

Accuracy on test data: 0.82


In [138]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

# Assuming the column 'gender' represents the sensitive attribute
# Replace 'gender' with the name of the column you want to evaluate bias on

sensitive_column = 'gender'
column_index = df.columns.get_loc(sensitive_column)

# Create a BinaryLabelDataset

privileged_groups = [{sensitive_column: 2}]  # 1 is the privileged
unprivileged_groups = [{sensitive_column: 1,sensitive_column:3}]  # 2 is the unprivileged
bld = BinaryLabelDataset(df=df, label_names=['decision'], protected_attribute_names=[sensitive_column],
                        unprivileged_protected_attributes=[unprivileged_groups])

# Compute fairness metric on original training dataset
metric_orig = BinaryLabelDatasetMetric(bld, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print('Original fairness metric:', metric_orig.mean_difference())

# Transform the original dataset to a fair representation
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
bld_transformed = RW.fit_transform(bld)

# Compute fairness metric on the transformed dataset
metric_modification = BinaryLabelDatasetMetric(bld_transformed, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print('Transformed fairness metric:', metric_modification.mean_difference())


Original fairness metric: 0.026344484081577746
Transformed fairness metric: 0.0


In [139]:
X_train_rw = bld_transformed.features
y_train_rw = bld_transformed.labels.ravel()

clf_rw = SVC(kernel='rbf', C=1)
clf_rw.fit(X_train_rw, y_train_rw)

y_pred_rw = clf_rw.predict(X_test)
print('Accuracy on test data after reweighing:', accuracy_score(y_test, y_pred_rw))

Accuracy on test data after reweighing: 0.8575


Nationality as sensitive information

In [140]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



df=pd.read_csv('./recruitmentdataset-2022-1.3.csv')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

df['gender'] = df['gender'].map({'male': 1, 'female': 2, 'other': 3})
df['nationality']=df['nationality'].map({'Dutch': 1, 'Belgian': 2,'German':3})


# Replace boolean values with 0 and 1
df = pd.get_dummies(df.drop('Id',axis=1))

df = df.replace({False: 0, True: 1})

# Min-max scaling of columns age, ind-university_grade and ind-languages
df['age'] = (df['age'] - df['age'].min()) / (df['age'].max() - df['age'].min())
df['ind-university_grade'] = (df['ind-university_grade'] - df['ind-university_grade'].min()) / (df['ind-university_grade'].max() - df['ind-university_grade'].min())
df['ind-languages'] = (df['ind-languages'] - df['ind-languages'].min()) / (df['ind-languages'].max() - df['ind-languages'].min())

X = df.drop(['decision'], axis=1)
y = df['decision']

# One-hot encode the target variable (one column for each class)
y = y.replace({False: 0, True: 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [141]:

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

clf = SVC(kernel='rbf', C=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print('Accuracy on test data:', accuracy_score(y_test, y_pred))

Accuracy on test data: 0.82


In [142]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

# Assuming the column 'gender' represents the sensitive attribute
# Replace 'gender' with the name of the column you want to evaluate bias on

sensitive_column = 'nationality'
column_index = df.columns.get_loc(sensitive_column)

# Create a BinaryLabelDataset

privileged_groups = [{sensitive_column: 1}]  # 1 is the privileged
unprivileged_groups = [{sensitive_column: 3,sensitive_column:2}]  # 2 is the unprivileged
bld = BinaryLabelDataset(df=df, label_names=['decision'], protected_attribute_names=[sensitive_column],
                        unprivileged_protected_attributes=[unprivileged_groups])

# Compute fairness metric on original training dataset
metric_orig = BinaryLabelDatasetMetric(bld, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print('Original fairness metric:', metric_orig.mean_difference())

# Transform the original dataset to a fair representation
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
bld_transformed = RW.fit_transform(bld)

# Compute fairness metric on the transformed dataset
metric_modification = BinaryLabelDatasetMetric(bld_transformed, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print('Transformed fairness metric:', metric_modification.mean_difference())


Original fairness metric: -0.00984228194267206
Transformed fairness metric: -1.6653345369377348e-16


In [143]:
X_train_rw = bld_transformed.features
y_train_rw = bld_transformed.labels.ravel()

clf_rw = SVC(kernel='rbf', C=1)
clf_rw.fit(X_train_rw, y_train_rw)

y_pred_rw = clf_rw.predict(X_test)
print('Accuracy on test data after reweighing:', accuracy_score(y_test, y_pred_rw))

Accuracy on test data after reweighing: 0.8575


Both at once

In [144]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



df=pd.read_csv('./recruitmentdataset-2022-1.3.csv')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

df['gender'] = df['gender'].map({'male': 1, 'female': 2, 'other': 3})
df['nationality']=df['nationality'].map({'Dutch': 1, 'Belgian': 2,'German':3})


# Replace boolean values with 0 and 1
df = pd.get_dummies(df.drop('Id',axis=1))

df = df.replace({False: 0, True: 1})

# Min-max scaling of columns age, ind-university_grade and ind-languages
df['age'] = (df['age'] - df['age'].min()) / (df['age'].max() - df['age'].min())
df['ind-university_grade'] = (df['ind-university_grade'] - df['ind-university_grade'].min()) / (df['ind-university_grade'].max() - df['ind-university_grade'].min())
df['ind-languages'] = (df['ind-languages'] - df['ind-languages'].min()) / (df['ind-languages'].max() - df['ind-languages'].min())

X = df.drop(['decision'], axis=1)
y = df['decision']

# One-hot encode the target variable (one column for each class)
y = y.replace({False: 0, True: 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [145]:

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

clf = SVC(kernel='rbf', C=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print('Accuracy on test data:', accuracy_score(y_test, y_pred))

Accuracy on test data: 0.82


In [160]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

# Assuming the column 'gender' represents the sensitive attribute
# Replace 'gender' with the name of the column you want to evaluate bias on

sensitive_column = ['gender','nationality']


# Create a BinaryLabelDataset

privileged_groups = [{'gender': 1,'nationality': 1}]  # 1 is the privileged
unprivileged_groups = [{'gender': 2,'gender':3},{'nationality': 2,'nationality': 3}]  # 2 is the unprivileged
bld = BinaryLabelDataset(df=df, label_names=['decision'], protected_attribute_names=['gender','nationality'],
                        unprivileged_protected_attributes=[unprivileged_groups])

# Compute fairness metric on original training dataset
metric_orig = BinaryLabelDatasetMetric(bld, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print('Original fairness metric:', metric_orig.mean_difference())

# Transform the original dataset to a fair representation
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
bld_transformed = RW.fit_transform(bld)

# Compute fairness metric on the transformed dataset
metric_modification = BinaryLabelDatasetMetric(bld_transformed, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print('Transformed fairness metric:', metric_modification.mean_difference())


Original fairness metric: -0.04329662899642617
Transformed fairness metric: -5.551115123125783e-17


In [161]:
X_train_rw = bld_transformed.features
y_train_rw = bld_transformed.labels.ravel()

clf_rw = SVC(kernel='rbf', C=1)
clf_rw.fit(X_train_rw, y_train_rw)

y_pred_rw = clf_rw.predict(X_test)
print('Accuracy on test data after reweighing:', accuracy_score(y_test, y_pred_rw))

Accuracy on test data after reweighing: 0.8575
